In [1]:
import os
import torch

# --- Main Paths ---
BASE_MODEL = "facebook/mbart-large-50"
DATA_PATH = "../../Dataset/new_large_CNN_dataset.csv" 
MODEL_OUTPUT_DIR = "mbart-large-50-cnn-summarizer-v15"
FINAL_SAVE_PATH = os.path.join(MODEL_OUTPUT_DIR, "final_model")

# --- Training Hyperparameters ---
LEARNING_RATE = 2e-5
NUM_EPOCHS = 4
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 8
WEIGHT_DECAY = 0.1 # Regularization to prevent overfitting

# --- Evaluation & Generation ---
METRIC_FOR_BEST_MODEL = "eval_bleurt_f1"
MAX_INPUT_LENGTH = 1024
MAX_SUMMARY_LENGTH = 256
EVAL_BEAMS = 5

# --- Compute ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import unicodedata
import pandas as pd
from datasets import Dataset, DatasetDict

def sanitize_text(text: str) -> str:
    """Basic text cleanup."""
    if not isinstance(text, str): 
        return ""
    return text.replace('""', '"').strip()

def normalize_text(text: str) -> str:
    """Normalize unicode characters."""
    if not isinstance(text, str): 
        return ""
    return ' '.join(unicodedata.normalize('NFKC', text).split())

def load_and_prep_dataset(data_path: str) -> DatasetDict:
    """Loads, cleans, formats, and splits the dataset."""
    
    # Load and clean
    df = pd.read_csv(data_path, engine='python', on_bad_lines='skip')
    df.dropna(subset=['raw_news_article', 'english_summary', 'hindi_summary'], inplace=True)
    
    for col in ['raw_news_article', 'english_summary', 'hindi_summary']:
        df[col] = df[col].apply(sanitize_text).apply(normalize_text)
    
    raw_dataset = Dataset.from_pandas(df)

    # Format for multilingual training
    processed_dataset = raw_dataset.map(
        _format_dataset_mbart, 
        batched=True, 
        remove_columns=raw_dataset.column_names
    )
    
    # Split
    train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)
    return DatasetDict({
        'train': train_test_split['train'],
        'test': train_test_split['test']
    })

def _format_dataset_mbart(batch):
    """Duplicates each article for its English and Hindi summary."""
    inputs, targets, langs = [], [], []
    for article, eng_summary, hin_summary in zip(
        batch['raw_news_article'], batch['english_summary'], batch['hindi_summary']
    ):
        if isinstance(article, str) and article:
            inputs.append(article)
            targets.append(eng_summary)
            langs.append("en_XX")
            
            inputs.append(article)
            targets.append(hin_summary)
            langs.append("hi_IN")
            
    return {'article': inputs, 'summary': targets, 'target_lang': langs}

def tokenize_function(examples, tokenizer, max_input_len, max_summary_len):
    """Tokenizes articles (inputs) and summaries (labels)."""
    
    tokenizer.src_lang = "en_XX"
    model_inputs = tokenizer(
        examples['article'], 
        max_length=max_input_len, 
        truncation=True
    )
    
    labels_batch = []
    for i in range(len(examples['summary'])):
        tokenizer.tgt_lang = examples['target_lang'][i]
        labels = tokenizer(
            text_target=examples['summary'][i], 
            max_length=max_summary_len, 
            truncation=True
        )
        labels_batch.append(labels['input_ids'])
    
    model_inputs["labels"] = labels_batch
    return model_inputs

In [3]:
import evaluate
import numpy as np

# Initialize metrics globally to avoid reloading
rouge_metric = evaluate.load("rouge")
bleurt_metric = evaluate.load("bleurt", "bleurt-20")

def compute_metrics(eval_pred, tokenizer):
    """Decodes predictions and computes ROUGE and BLEURT scores."""
    predictions, labels = eval_pred
    
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    # Compute BLEURT
    bleurt_result = bleurt_metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    result = {
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
        "bleurt_f1": np.mean(bleurt_result["scores"])
    }
    
    return {k: round(v, 4) for k, v in result.items()}


INFO:tensorflow:Reading checkpoint C:\Users\admin\.cache\huggingface\metrics\bleurt\bleurt-20\downloads\extracted\8db8856a80394ae84b010e83ab663d4a3ccfa244ce3d0dbe00143f73e65ff123\BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: C:\Users\admin\.cache\huggingface\metrics\bleurt\bleurt-20\downloads\extracted\8db8856a80394ae84b010e83ab663d4a3ccfa244ce3d0dbe00143f73e65ff123\BLEURT-20\sent_piece.model.
INFO:tensorflow:Sentenc

INFO:tensorflow:BLEURT initialized.


In [6]:
import logging
import os
from functools import partial
from datetime import datetime
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Import custom modules
import config
import data_utils
import metrics_utils

def setup_logging():
    """Configures logging to file and stream."""
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    log_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"training_log_v15_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    )
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] - %(message)s",
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )
    logging.info(f"--- Starting mBART v15 Training from Scratch ---")
    logging.info(f"Logging to: {log_filename}")

def main():
    setup_logging()

    # 1. Load and Prepare Data
    logging.info(f"Loading and processing data from: {config.DATA_PATH}")
    tokenized_datasets = data_utils.load_and_prep_dataset(config.DATA_PATH)
    logging.info(f"Dataset split: {len(tokenized_datasets['train'])} train, {len(tokenized_datasets['test'])} test")

    # 2. Load Base Model and Tokenizer
    logging.info(f"Loading base model and tokenizer from: {config.BASE_MODEL}")
    tokenizer = MBart50TokenizerFast.from_pretrained(config.BASE_MODEL)
    model = MBartForConditionalGeneration.from_pretrained(config.BASE_MODEL, use_safetensors=True)
    
    # 3. Tokenize Datasets
    logging.info("Tokenizing datasets...")
    tokenized_datasets = tokenized_datasets.map(
        partial(
            data_utils.tokenize_function,
            tokenizer=tokenizer,
            max_input_len=config.MAX_INPUT_LENGTH,
            max_summary_len=config.MAX_SUMMARY_LENGTH
        ),
        batched=True,
        remove_columns=['article', 'summary', 'target_lang']
    )
    
    # 4. Define Training Arguments
    logging.info("Configuring training arguments...")
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        
        # Hyperparameters
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        weight_decay=config.WEIGHT_DECAY,
        
        # Logging and Saving (per epoch, as requested)
        logging_dir=os.path.join(config.MODEL_OUTPUT_DIR, "logs"),
        logging_strategy="epoch",
        eval_strategy="epoch", # Corrected from evaluation_strategy
        save_strategy="epoch",
        
        # We will run a separate script to find the best model
        load_best_model_at_end=False,
        save_total_limit=config.NUM_EPOCHS, # Save all checkpoints
        
        # Other settings
        predict_with_generate=True,
        fp16=config.DEVICE.type == 'cuda',
        report_to="tensorboard",
        generation_max_length=config.MAX_SUMMARY_LENGTH,
        generation_num_beams=config.EVAL_BEAMS,
    )
    
    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=partial(metrics_utils.compute_metrics, tokenizer=tokenizer),
    )

    # 6. Start Training
    logging.info(f"--- Training started for {config.NUM_EPOCHS} epochs ---")
    trainer.train()
    logging.info("--- Training finished successfully ---")
    logging.info(f"Checkpoints and logs are saved in: {config.MODEL_OUTPUT_DIR}")
    logging.info("Run 'evaluate.py' to find the best model and save it.")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/9223 [00:00<?, ? examples/s]

Map:   0%|          | 0/16601 [00:00<?, ? examples/s]

Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_27548\1931657635.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [7]:
import logging
import os
from functools import partial
from datetime import datetime
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Import custom modules
import config
import data_utils
import metrics_utils

def setup_logging():
    """Configures logging to file and stream."""
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    log_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"training_log_v15_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    )
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] - %(message)s",
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )
    logging.info(f"--- Starting mBART v15 Training from Scratch ---")
    logging.info(f"Logging to: {log_filename}")

def main():
    setup_logging()

    # 1. Load and Prepare Data
    logging.info(f"Loading and processing data from: {config.DATA_PATH}")
    tokenized_datasets = data_utils.load_and_prep_dataset(config.DATA_PATH)
    logging.info(f"Dataset split: {len(tokenized_datasets['train'])} train, {len(tokenized_datasets['test'])} test")

    # 2. Load Base Model and Tokenizer
    logging.info(f"Loading base model and tokenizer from: {config.BASE_MODEL}")
    tokenizer = MBart50TokenizerFast.from_pretrained(config.BASE_MODEL)
    model = MBartForConditionalGeneration.from_pretrained(config.BASE_MODEL, use_safetensors=True)
    
    # 3. Tokenize Datasets
    logging.info("Tokenizing datasets...")
    tokenized_datasets = tokenized_datasets.map(
        partial(
            data_utils.tokenize_function,
            tokenizer=tokenizer,
            max_input_len=config.MAX_INPUT_LENGTH,
            max_summary_len=config.MAX_SUMMARY_LENGTH
        ),
        batched=True,
        remove_columns=['article', 'summary', 'target_lang']
    )
    
    # 4. Define Training Arguments
    logging.info("Configuring training arguments...")
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        
        # Hyperparameters
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        weight_decay=config.WEIGHT_DECAY,
        
        # Logging and Saving (per epoch, as requested)
        logging_dir=os.path.join(config.MODEL_OUTPUT_DIR, "logs"),
        logging_strategy="epoch",
        eval_strategy="epoch", # Corrected from evaluation_strategy
        save_strategy="epoch",
        
        # We will run a separate script to find the best model
        load_best_model_at_end=False,
        save_total_limit=config.NUM_EPOCHS, # Save all checkpoints
        
        # Other settings
        predict_with_generate=True,
        fp16=config.DEVICE.type == 'cuda',
        report_to="tensorboard",
        generation_max_length=config.MAX_SUMMARY_LENGTH,
        generation_num_beams=config.EVAL_BEAMS,
    )
    
    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=partial(metrics_utils.compute_metrics, tokenizer=tokenizer),
    )

    # 6. Start Training
    logging.info(f"--- Training started for {config.NUM_EPOCHS} epochs ---")
    trainer.train()
    logging.info("--- Training finished successfully ---")
    logging.info(f"Checkpoints and logs are saved in: {config.MODEL_OUTPUT_DIR}")
    logging.info("Run 'evaluate.py' to find the best model and save it.")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/9223 [00:00<?, ? examples/s]

Map:   0%|          | 0/16601 [00:00<?, ? examples/s]

Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_27548\1931657635.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [9]:
import logging
import os
from functools import partial
from datetime import datetime
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Import custom modules
import config
import data_utils
import metrics_utils

def setup_logging():
    """Configures logging to file and stream."""
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    log_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"training_log_v15_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    )
    # Use force=True to re-configure logging in a notebook environment
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] - %(message)s",
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ],
        force=True
    )
    logging.info(f"--- Starting mBART v15 Training ---")
    logging.info(f"Logging to: {log_filename}")

def main():
    setup_logging()

    # 1. Load and Prepare Data
    logging.info(f"Loading and processing data from: {config.DATA_PATH}")
    tokenized_datasets = data_utils.load_and_prep_dataset(config.DATA_PATH)
    logging.info(f"Dataset split: {len(tokenized_datasets['train'])} train, {len(tokenized_datasets['test'])} test")

    # 2. Load Base Model and Tokenizer
    logging.info(f"Loading base model and tokenizer from: {config.BASE_MODEL}")
    tokenizer = MBart50TokenizerFast.from_pretrained(config.BASE_MODEL)
    model = MBartForConditionalGeneration.from_pretrained(config.BASE_MODEL, use_safetensors=True)
    
    # 3. Tokenize Datasets
    logging.info("Tokenizing datasets...")
    tokenized_datasets = tokenized_datasets.map(
        partial(
            data_utils.tokenize_function,
            tokenizer=tokenizer,
            max_input_len=config.MAX_INPUT_LENGTH,
            max_summary_len=config.MAX_SUMMARY_LENGTH
        ),
        batched=True,
        remove_columns=['article', 'summary', 'target_lang']
    )
    
    # 4. Define Training Arguments
    logging.info("Configuring training arguments...")
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        
        # Hyperparameters
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        weight_decay=config.WEIGHT_DECAY,
        
        # --- LOGGING FIX ---
        logging_dir=os.path.join(config.MODEL_OUTPUT_DIR, "logs"),
        logging_strategy="steps",   # Log every N steps
        logging_steps=150,          # Log training loss every 100 steps
        eval_strategy="epoch",      # Run full evaluation every epoch
        save_strategy="epoch",
        # ---------------------
        
        load_best_model_at_end=False,
        save_total_limit=config.NUM_EPOCHS, # Save all checkpoints
        
        # Other settings
        predict_with_generate=True,
        fp16=config.DEVICE.type == 'cuda',
        report_to="tensorboard",
        generation_max_length=config.MAX_SUMMARY_LENGTH,
        generation_num_beams=config.EVAL_BEAMS,
    )
    
    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=partial(metrics_utils.compute_metrics, tokenizer=tokenizer),
    )

    # 6. Start Training
    logging.info(f"--- Training started for {config.NUM_EPOCHS} epochs ---")
    trainer.train()
    logging.info("--- Training finished successfully ---")
    logging.info(f"Checkpoints and logs are saved in: {config.MODEL_OUTPUT_DIR}")
    logging.info("Run 'evaluate.py' to find the best model and save it.")

if __name__ == "__main__":
    main()
    

2025-11-13 18:21:14,270 [INFO] - --- Starting mBART v15 Training ---
2025-11-13 18:21:14,275 [INFO] - Logging to: mbart-large-50-cnn-summarizer-v15\training_log_v15_2025-11-13_18-21-14.log
2025-11-13 18:21:14,276 [INFO] - Loading and processing data from: ../../Dataset/new_large_CNN_dataset.csv


Map:   0%|          | 0/9223 [00:00<?, ? examples/s]

2025-11-13 18:21:17,640 [INFO] - Dataset split: 16601 train, 1845 test
2025-11-13 18:21:17,642 [INFO] - Loading base model and tokenizer from: facebook/mbart-large-50
2025-11-13 18:21:26,440 [INFO] - Tokenizing datasets...


Map:   0%|          | 0/16601 [00:00<?, ? examples/s]

Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

2025-11-13 18:22:12,913 [INFO] - Configuring training arguments...
C:\Users\admin\AppData\Local\Temp\ipykernel_27548\1042724579.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
2025-11-13 18:22:14,899 [INFO] - --- Training started for 4 epochs ---


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import logging
import os
from functools import partial
from datetime import datetime
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Import custom modules
import config
import data_utils
import metrics_utils

def setup_logging():
    """Configures logging to file and stream."""
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    log_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"training_log_v15_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    )
    # Use force=True to re-configure logging in a notebook environment
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] - %(message)s",
        handlers=[
            logging.FileHandler(log_filename)
            # REMOVED the StreamHandler(). This will stop the conflict
            # and allow the Trainer to log to the console by default.
        ],
        force=True
    )
    logging.info(f"--- Starting mBART v15 Training ---")
    logging.info(f"Logging to: {log_filename}")

def main():
    setup_logging()

    # 1. Load and Prepare Data
    logging.info(f"Loading and processing data from: {config.DATA_PATH}")
    final_datasets = data_utils.load_and_prep_dataset(config.DATA_PATH)
    logging.info(f"Dataset split: {len(final_datasets['train'])} train, {len(final_datasets['test'])} test")

    # 2. Load Base Model and Tokenizer
    logging.info(f"Loading base model and tokenizer from: {config.BASE_MODEL}")
    tokenizer = MBart50TokenizerFast.from_pretrained(config.BASE_MODEL)
    # Use safetensors=True to avoid the torch.load vulnerability error
    model = MBartForConditionalGeneration.from_pretrained(config.BASE_MODEL, use_safetensors=True)
    
    # 3. Tokenize Datasets
    logging.info("Tokenizing datasets...")
    # Use functools.partial to pass static arguments to the map function
    _tokenize_fn = partial(
        data_utils.tokenize_function,
        tokenizer=tokenizer,
        max_input_len=config.MAX_INPUT_LENGTH,
        max_summary_len=config.MAX_SUMMARY_LENGTH
    )
    tokenized_datasets = final_datasets.map(
        _tokenize_fn,
        batched=True,
        remove_columns=['article', 'summary', 'target_lang']
    )
    
    # 4. Define Training Arguments
    logging.info("Configuring training arguments...")
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        
        # Hyperparameters
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        weight_decay=config.WEIGHT_DECAY,
        
        # --- LOGGING FIX ---
        logging_dir=os.path.join(config.MODEL_OUTPUT_DIR, "logs"),
        logging_strategy="steps",   # Log every N steps
        logging_steps=50,           # Log training loss every 50 steps
        log_level="info",           # This will now correctly print to the notebook
        eval_strategy="epoch",      # Run full evaluation every epoch
        save_strategy="epoch",
        # ---------------------
        
        load_best_model_at_end=False, # We run a separate script to evaluate
        save_total_limit=config.NUM_EPOCHS, # Save all checkpoints
        
        # Other settings
        predict_with_generate=True,
        fp16=config.DEVICE.type == 'cuda',
        report_to="tensorboard",
        generation_max_length=config.MAX_SUMMARY_LENGTH,
        generation_num_beams=config.EVAL_BEAMS,
    )
    
    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    # Use partial to pass the tokenizer to the metrics function
    _compute_metrics_fn = partial(metrics_utils.compute_metrics, tokenizer=tokenizer)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=_compute_metrics_fn,
    )

    # 6. Start Training
    logging.info(f"--- Training started for {config.NUM_EPOCHS} epochs ---")
    trainer.train()
    logging.info("--- Training finished successfully ---")
    logging.info(f"Checkpoints and logs are saved in: {config.MODEL_OUTPUT_DIR}")
    logging.info("Run 'evaluate.py' to find the best model and save it.")

if __name__ == "__main__":
    main()

Evaluation and Saving the best model

In [ ]:
import os
import json
import shutil
import logging
import config 
import logging_utils

def find_best_checkpoint():
    """
    Parses trainer_state.json files to find the best checkpoint
    and copies it to the final_model directory.
    """
    log_path = logging_utils.setup_logging(config.MODEL_OUTPUT_DIR, "evaluation_log_v15")
    print(f"--- Starting Evaluation & Save Script ---")
    print(f"Logging all output to: {log_path}\n")
    logging.info("--- Starting Post-Training Evaluation & Save Script ---")
    
    logging.info(f"Using '{config.METRIC_FOR_BEST_MODEL}' as the key metric.")
    print(f"Using '{config.METRIC_FOR_BEST_MODEL}' as the key metric.")
    
    best_metric_value = -1.0 
    best_checkpoint_path = None
    all_results = []

    checkpoint_dirs = [
        d for d in os.listdir(config.MODEL_OUTPUT_DIR) 
        if d.startswith("checkpoint-") and os.path.isdir(os.path.join(config.MODEL_OUTPUT_DIR, d))
    ]
    
    if not checkpoint_dirs:
        logging.error(f"No checkpoint directories found in {config.MODEL_OUTPUT_DIR}")
        print(f"Error: No checkpoint directories found in {config.MODEL_OUTPUT_DIR}")
        return

    logging.info(f"Found {len(checkpoint_dirs)} checkpoints to evaluate.")
    print(f"Found {len(checkpoint_dirs)} checkpoints to evaluate.")

    for chkpt_dir in checkpoint_dirs:
        state_path = os.path.join(config.MODEL_OUTPUT_DIR, chkpt_dir, "trainer_state.json")
        if not os.path.exists(state_path):
            logging.warning(f"No trainer_state.json found in {chkpt_dir}, skipping.")
            continue

        with open(state_path, "r") as f:
            state = json.load(f)
        
        eval_log = None
        for log in reversed(state["log_history"]):
            if config.METRIC_FOR_BEST_MODEL in log:
                eval_log = log
                break
        
        if eval_log:
            all_results.append(eval_log)
            metric_value = eval_log[config.METRIC_FOR_BEST_MODEL]
            
            if metric_value > best_metric_value:
                best_metric_value = metric_value
                best_checkpoint_path = os.path.join(config.MODEL_OUTPUT_DIR, chkpt_dir)
                logging.info(f"*** New best checkpoint: {chkpt_dir} ({config.METRIC_FOR_BEST_MODEL}: {metric_value}) ***")
                print(f"*** New best checkpoint: {chkpt_dir} ({config.METRIC_FOR_BEST_MODEL}: {metric_value}) ***")
        else:
            logging.warning(f"No evaluation metrics found in {state_path}.")

    # Log summary table
    summary_header = "\n" + "="*80 + "\n" + "--- FINAL EVALUATION SUMMARY ---".center(80) + "\n"
    table_header = f"{'Checkpoint':<20} | {'Step':<10} | {'Loss':<10} | {config.METRIC_FOR_BEST_MODEL:<12} | {'RougeL':<10}"
    summary_divider = "-" * len(table_header)
    
    logging.info(summary_header); print(summary_header)
    logging.info(table_header); print(table_header)
    logging.info(summary_divider); print(summary_divider)
    
    for log in sorted(all_results, key=lambda x: x['step']):
        name = f"checkpoint-{log['step']}"
        loss = log.get('eval_loss', 0.0)
        bleurt = log.get(config.METRIC_FOR_BEST_MODEL, 0.0)
        rougeL = log.get('eval_rougeL', 0.0)
        row = f"{name:<20} | {log['step']:<10} | {loss:<10.4f} | {bleurt:<12.4f} | {rougeL:<10.4f}"
        logging.info(row); print(row)
    
    logging.info("="*80 + "\n"); print("="*80 + "\n")

    # Save the best model
    if best_checkpoint_path:
        logging.info(f"Best model identified: {best_checkpoint_path}")
        print(f"Best model identified: {best_checkpoint_path}")
        
        if os.path.exists(config.FINAL_SAVE_PATH):
            logging.warning(f"Removing existing directory: {config.FINAL_SAVE_PATH}")
            print(f"Removing existing directory: {config.FINAL_SAVE_PATH}")
            shutil.rmtree(config.FINAL_SAVE_PATH)
            
        logging.info(f"Copying {best_checkpoint_path} to {config.FINAL_SAVE_PATH}...")
        print(f"Copying {best_checkpoint_path} to {config.FINAL_SAVE_PATH}...")
        shutil.copytree(best_checkpoint_path, config.FINAL_SAVE_PATH)
        
        logging.info("--- Best model saved successfully! ---")
        print("\n--- Best model saved successfully! ---")
        logging.info(f"You can now test it by running: python test.py")
        print(f"You can now test it by running: python test.py")
    else:
        logging.error("Could not determine the best model.")
        print("Error: Could not determine the best model.")

if __name__ == "__main__":
    try:
        find_best_checkpoint()
    except (KeyboardInterrupt):
        print("\n--- Evaluation Interrupted ---")
    except Exception as e:
        print(f"\n--- Evaluation FAILED --- \nError: {e}\nSee log file for details.")

Testing and Inferences

In [3]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap
import os
import config 
import logging_utils
import logging

model = None
tokenizer = None

def load_model():
    """Loads the final v15 model and tokenizer."""
    global model, tokenizer
    
    if not os.path.exists(config.FINAL_SAVE_PATH):
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        logging.error(f"Model path not found: {config.FINAL_SAVE_PATH}")
        print("Please run 'train.py' and 'evaluate.py' first.")
        return False

    print(f"Loading final model from: {config.FINAL_SAVE_PATH}...")
    logging.info(f"Loading final model from: {config.FINAL_SAVE_PATH}...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        print(f"--- Model v15 loaded successfully on {config.DEVICE} ---")
        logging.info(f"--- Model v15 loaded successfully on {config.DEVICE} ---")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        logging.error(f"Error loading model: {e}", exc_info=True)
        return False

def summarize_text(article_text: str):
    """Generates a high-quality summary."""
    if model is None or tokenizer is None:
        print("Model is not loaded. Please load the model first.")
        logging.error("summarize_text called before model was loaded.")
        return

    gen_kwargs = {
        "num_beams": 12,
        "length_penalty": 2.0,
        "repetition_penalty": 2.5,
        "no_repeat_ngram_size": 3,
        "do_sample": False,
        "early_stopping": True,
        "min_length": 30,
        "max_length": 250,
    }
    
    print("\n" + "="*80)
    print("SOURCE ARTICLE:")
    print(textwrap.fill(article_text, width=80))
    logging.info(f"--- Summarizing Source Article --- \n{article_text}\n")

    tokenizer.src_lang = "en_XX"
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True).to(config.DEVICE)

    # --- Generate English ---
    eng_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"],
        **gen_kwargs
    )
    english_summary = tokenizer.decode(eng_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED ENGLISH SUMMARY (v15):")
    print(textwrap.fill(english_summary, width=80))
    logging.info(f"--- Generated English Summary --- \n{english_summary}\n")

    # --- Generate Hindi ---
    hin_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        **gen_kwargs
    )
    hindi_summary = tokenizer.decode(hin_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED HINDI SUMMARY (v15):")
    print(textwrap.fill(hindi_summary, width=80)) 
    logging.info(f"--- Generated Hindi Summary --- \n{hindi_summary}\n")
    print("\n" + "="*80)

def main():
    log_path = logging_utils.setup_logging(config.MODEL_OUTPUT_DIR, "testing_log_v15")
    print(f"--- Starting Test Script ---")
    print(f"Logging all output to: {log_path}\n")
    logging.info("--- Starting Test Script ---")
    
    if load_model():
        article_to_test = """
        A major tech firm today unveiled its latest flagship smartphone, 
        featuring a revolutionary new camera system with 'periscope zoom' 
        technology. The device, which also boasts a foldable OLED display 
        and 5G connectivity, aims to redefine the premium mobile market. 
        Analysts are optimistic, noting that the innovative camera could be 
        a key differentiator in a crowded field. However, concerns remain 
        about the device's high price point, which exceeds $1,500, 
        potentially limiting its mass-market appeal despite the advanced features.
        """
        summarize_text(article_to_test)

if __name__ == "__main__":
    try:
        main()
    except (KeyboardInterrupt):
        print("\n--- Test Interrupted ---")
    except Exception as e:
        print(f"\n--- Test FAILED --- \nError: {e}\nSee log file for details.")

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\config.json
Model config MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "dtype": "float32",
  "early_stopping": null,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL

--- Starting Test Script ---
Logging all output to: mbart-large-50-cnn-summarizer-v15\testing_log_v15_2025-11-14_07-35-31.log

Loading final model from: mbart-large-50-cnn-summarizer-v15\final_model...


loading weights file mbart-large-50-cnn-summarizer-v15\final_model\model.safetensors
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "pad_token_id": 1
}

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": [
    2
  ],
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Could not locate the custom_generate/generate.py inside mbart-large-50-cnn-summarizer-v15\final_model.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


--- Model v15 loaded successfully on cuda ---

SOURCE ARTICLE:
         A major tech firm today unveiled its latest flagship smartphone,
featuring a revolutionary new camera system with 'periscope zoom'
technology. The device, which also boasts a foldable OLED display          and
5G connectivity, aims to redefine the premium mobile market.          Analysts
are optimistic, noting that the innovative camera could be          a key
differentiator in a crowded field. However, concerns remain          about the
device's high price point, which exceeds $1,500,          potentially limiting
its mass-market appeal despite the advanced features.

GENERATED ENGLISH SUMMARY (v15):
A major tech firm has unveiled its latest flagship smartphone, featuring a
revolutionary new camera system with 'periscope zoom' technology. This device,
equipped with a foldable OLED display and 5G connectivity, aims to redefine the
premium mobile market. Analysts are optimistic, noting that the innovative
camera cou

In [4]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap
import os
import config 

model = None
tokenizer = None

def load_model():
    """Loads the final v15 model and tokenizer."""
    global model, tokenizer
    
    if not os.path.exists(config.FINAL_SAVE_PATH):
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        print("Please run 'train.py' and 'evaluate.py' first.")
        return False

    print(f"Loading final model from: {config.FINAL_SAVE_PATH}...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        print(f"--- Model v15 loaded successfully on {config.DEVICE} ---")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        return False

def summarize_text(article_text: str):
    """Generates a high-quality Hindi summary."""
    if model is None or tokenizer is None:
        print("Model is not loaded. Please load the model first.")
        return

    # Generation parameters are important for quality
    gen_kwargs = {
        "num_beams": 12,
        "length_penalty": 2.0,
        "repetition_penalty": 2.5,
        "no_repeat_ngram_size": 3,
        "do_sample": False,
        "early_stopping": True,
        "min_length": 30,
        "max_length": 250,
    }
    
    print("\n" + "="*80)
    print("SOURCE ARTICLE:")
    print(textwrap.fill(article_text, width=80))

    tokenizer.src_lang = "en_XX"
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True).to(config.DEVICE)

    # --- Generate Hindi Summary ---
    hin_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        **gen_kwargs
    )
    hindi_summary = tokenizer.decode(hin_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED HINDI SUMMARY (v15):")
    print(textwrap.fill(hindi_summary, width=80)) 
    print("\n" + "="*80)

def main():
    print(f"--- Starting Test Script ---")
    
    if load_model():
        article_to_test = """
        A major tech firm today unveiled its latest flagship smartphone, 
        featuring a revolutionary new camera system with 'periscope zoom' 
        technology. The device, which also boasts a foldable OLED display 
        and 5G connectivity, aims to redefine the premium mobile market. 
        Analysts are optimistic, noting that the innovative camera could be 
        a key differentiator in a crowded field. However, concerns remain 
        about the device's high price point, which exceeds $1,500, 
        potentially limiting its mass-market appeal despite the advanced features.
        """
        summarize_text(article_to_test)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n--- Test Interrupted by user ---")
    except Exception as e:
        print(f"\n--- Test FAILED with an unexpected error: {e}")

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\config.json
Model config MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "dtype": "float32",
  "early_stopping": null,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL

--- Starting Test Script ---
Loading final model from: mbart-large-50-cnn-summarizer-v15\final_model...


loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": [
    2
  ],
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Could not locate the custom_generate/generate.py inside mbart-large-50-cnn-summarizer-v15\final_model.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


--- Model v15 loaded successfully on cuda ---

SOURCE ARTICLE:
         A major tech firm today unveiled its latest flagship smartphone,
featuring a revolutionary new camera system with 'periscope zoom'
technology. The device, which also boasts a foldable OLED display          and
5G connectivity, aims to redefine the premium mobile market.          Analysts
are optimistic, noting that the innovative camera could be          a key
differentiator in a crowded field. However, concerns remain          about the
device's high price point, which exceeds $1,500,          potentially limiting
its mass-market appeal despite the advanced features.

GENERATED HINDI SUMMARY (v15):
एक प्रमुख टेक्स्ट फर्म ने अपनी नवीनतम फ्लैगशिप स्मार्टफोन का अनावरण किया है, जो
'पेरिस्कॉज़' तकनीक के साथ एक क्रांतिकारी नए कैमरे सिस्टम और foldable OLED
डिस्प्ले से चिह्नित है। यह स्मार्टफ़ोन प्रीमियम मोबाइल बाजार को फिर से परिभाषित
करने का लक्ष्य रखता है, और विश्लेषक आशावादी हैं कि यह एक भीड़भाड़ वाले क्षेत्र
में एक प

In [5]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap
import os
import config 

model = None
tokenizer = None

def load_model():
    """Loads the final v15 model and tokenizer."""
    global model, tokenizer
    
    if not os.path.exists(config.FINAL_SAVE_PATH):
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        print("Please run 'train.py' and 'evaluate.py' first.")
        return False

    # Minimal loading message
    print("Loading final model...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        print("--- Model v15 Loaded ---")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        return False

def summarize_text(article_text: str):
    """Generates a high-quality Hindi summary."""
    if model is None or tokenizer is None:
        print("Model is not loaded. Please load the model first.")
        return

    # Generation parameters are important for quality
    gen_kwargs = {
        "num_beams": 12,
        "length_penalty": 2.0,
        "repetition_penalty": 2.5,
        "no_repeat_ngram_size": 3,
        "do_sample": False,
        "early_stopping": True,
        "min_length": 30,
        "max_length": 250,
    }
    
    print("\n" + "="*80)
    print("SOURCE ARTICLE:")
    print(textwrap.fill(article_text, width=80))

    tokenizer.src_lang = "en_XX"
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True).to(config.DEVICE)

    # --- Generate Hindi Summary ---
    hin_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        **gen_kwargs
    )
    hindi_summary = tokenizer.decode(hin_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED HINDI SUMMARY (v15):")
    print(textwrap.fill(hindi_summary, width=80)) 
    print("\n" + "="*80)

def main():
    if load_model():
        article_to_test = """
        A major tech firm today unveiled its latest flagship smartphone, 
        featuring a revolutionary new camera system with 'periscope zoom' 
        technology. The device, which also boasts a foldable OLED display 
        and 5G connectivity, aims to redefine the premium mobile market. 
        Analysts are optimistic, noting that the innovative camera could be 
        a key differentiator in a crowded field. However, concerns remain 
        about the device's high price point, which exceeds $1,500, 
        potentially limiting its mass-market appeal despite the advanced features.
        """
        summarize_text(article_to_test)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n--- Test Interrupted by user ---")
    except Exception as e:
        print(f"\n--- Test FAILED with an unexpected error: {e}")

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\config.json
Model config MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "dtype": "float32",
  "early_stopping": null,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL

Loading final model...


loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": [
    2
  ],
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Could not locate the custom_generate/generate.py inside mbart-large-50-cnn-summarizer-v15\final_model.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


--- Model v15 Loaded ---

SOURCE ARTICLE:
         A major tech firm today unveiled its latest flagship smartphone,
featuring a revolutionary new camera system with 'periscope zoom'
technology. The device, which also boasts a foldable OLED display          and
5G connectivity, aims to redefine the premium mobile market.          Analysts
are optimistic, noting that the innovative camera could be          a key
differentiator in a crowded field. However, concerns remain          about the
device's high price point, which exceeds $1,500,          potentially limiting
its mass-market appeal despite the advanced features.

GENERATED HINDI SUMMARY (v15):
एक प्रमुख टेक्स्ट फर्म ने अपनी नवीनतम फ्लैगशिप स्मार्टफोन का अनावरण किया है, जो
'पेरिस्कॉज़' तकनीक के साथ एक क्रांतिकारी नए कैमरे सिस्टम और foldable OLED
डिस्प्ले से चिह्नित है। यह स्मार्टफ़ोन प्रीमियम मोबाइल बाजार को फिर से परिभाषित
करने का लक्ष्य रखता है, और विश्लेषक आशावादी हैं कि यह एक भीड़भाड़ वाले क्षेत्र
में एक प्रमुख विनिर्देशक बन स

In [ ]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap
import os
import config

model = None
tokenizer = None
gen_kwargs = {}

def load_model_and_settings():
    """Loads the v15 model, tokenizer, and high-quality generation parameters."""
    global model, tokenizer, gen_kwargs
    
    if model is not None:
        print("Model is already loaded.")
        return True

    if not os.path.exists(config.FINAL_SAVE_PATH):
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        print("Please run 'train.py' and 'evaluate.py' first.")
        return False

    print("Loading final model...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        
        gen_kwargs = {
            "num_beams": 12,
            "length_penalty": 2.0,
            "repetition_penalty": 2.5,
            "no_repeat_ngram_size": 3,
            "do_sample": False,
            "early_stopping": True,
            "min_length": 30,
            "max_length": 250,
        }
        
        print(f"--- Model v15 Loaded on {config.DEVICE} ---")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        return False

def summarize_hindi(article_text: str):
    """Generates a high-quality Hindi summary for the given text."""
    if model is None or tokenizer is None:
        print("Model is not loaded. Please run the setup cell (Cell 1) first.")
        return

    print("\n" + "="*80)
    print("SOURCE ARTICLE:")
    print(textwrap.fill(article_text, width=80))

    tokenizer.src_lang = "en_XX"
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True).to(config.DEVICE)

    # Generate Hindi Summary
    hin_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        **gen_kwargs  
    )
    hindi_summary = tokenizer.decode(hin_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED HINDI SUMMARY (v15):")
    print(textwrap.fill(hindi_summary, width=80)) 
    print("\n" + "="*80)

load_model_and_settings()

In [7]:
import summarizer_tool

def main():
    if summarizer_tool.load_model_and_settings():
        
        articles_to_test = [
            # Article 1: Tech
            """
            A major tech firm today unveiled its latest flagship smartphone, 
            featuring a revolutionary new camera system with 'periscope zoom' 
            technology. The device, which also boasts a foldable OLED display 
            and 5G connectivity, aims to redefine the premium mobile market. 
            Analysts are optimistic, noting that the innovative camera could be 
            a key differentiator in a crowded field. However, concerns remain 
            about the device's high price point, which exceeds $1,500, 
            potentially limiting its mass-market appeal despite the advanced features.
            """,
            
            # Article 2: Finance
            """
            Global stock markets experienced a volatile week as central banks 
            around the world signaled a more aggressive stance on combating inflation. 
            The US Federal Reserve hinted at larger-than-expected interest rate hikes, 
            causing a sell-off in technology stocks and growth-oriented sectors. 
            Meanwhile, the European Central Bank is facing pressure to act as energy 
            prices continue to soar across the continent, impacting both consumer 
            spending and industrial production.
            """,
            
            # Article 3: Sports
            """
            India secured a decisive victory over Australia in the final match of the T20 
            series, winning by a margin of 35 runs in Bengaluru. Batting first, India posted 
            a competitive total of 198 for 4, thanks to a powerful half-century from captain 
            Suryakumar Yadav, who scored 78 off just 45 balls. In response, Australia's 
            chase faltered early as they lost key wickets to India's fast bowlers.
            """,
            
            # Article 4: Science
            """
            NASA's Artemis program achieved a major milestone this week as the Orion 
            spacecraft successfully completed its uncrewed flyby of the Moon and is now on 
            its return trajectory to Earth. The mission, Artemis I, is a critical test of 
            the agency's deep space exploration systems. The spacecraft's heat shield will 
            face its most extreme test during re-entry, when it will endure temperatures 
            of nearly 5,000 degrees Fahrenheit.
            """,

            # Article 5: Environment
            """
            A new international report has found that the rate of deforestation in the 
            Amazon rainforest accelerated by nearly 20% in the last year, reaching its 
            highest level in over a decade. The report attributes the surge to increased 
            illegal logging, agricultural expansion, and mining activities.
            """,

            # Article 6: Health
            """
            Researchers have published a landmark study detailing a new gene-editing 
            technique that shows promise in correcting genetic defects responsible for 
            certain inherited diseases. The method, using a modified CRISPR-Cas9 system, 
            demonstrated significantly higher precision and lower off-target mutations 
            in lab experiments compared to existing technologies.
            """,

            # Article 7: Politics
            """
            Diplomats from several nations met in Geneva to resume peace talks aimed at 
            resolving a long-standing regional conflict. The negotiations, which had been 
            stalled for months, were restarted following a recent de-escalation of 
            hostilities. The primary goal is to establish a lasting ceasefire.
            """,

            # Article 8: Entertainment
            """
            The highly anticipated sequel to a blockbuster science fiction film has 
            officially begun production, with the studio releasing the first on-set photo. 
            The image features the return of the original cast members alongside several new 
            additions. The movie is currently slated for a summer release next year.
            """,

            # Article 9: Automotive
            """
            A legacy automaker has announced a massive $50 billion investment into its 
            electric vehicle (EV) division, signaling a dramatic acceleration of its 
            transition away from internal combustion engines. The company plans to launch 15 
            new all-electric models over the next three years, including SUVs and a 
            pickup truck.
            """,

            # Article 10: Trade
            """
            A new trade agreement between two major economic blocs was signed this week, 
            aiming to reduce tariffs and streamline regulations across dozens of industries. 
            The pact, which covers everything from agricultural goods to digital services, 
            is the culmination of nearly five years of intense negotiations.
            """
        ]
        
        # Loop through and summarize each article
        for i, article in enumerate(articles_to_test, 1):
            print(f"\n\n--- SUMMARIZING ARTICLE {i}/{len(articles_to_test)} ---")
            summarizer_tool.summarize_hindi(article)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n--- Test Interrupted ---")

ModuleNotFoundError: No module named 'summarizer_tool'

In [8]:
import summarizer_tool

def main():
    # Load the model once
    if summarizer_tool.load_model_and_settings():
        
        # New article to test (Finance)
        article_to_test = """
        Global stock markets experienced a volatile week as central banks 
        around the world signaled a more aggressive stance on combating inflation. 
        The US Federal Reserve hinted at larger-than-expected interest rate hikes, 
        causing a sell-off in technology stocks and growth-oriented sectors. 
        Meanwhile, the European Central Bank is facing pressure to act as energy 
        prices continue to soar across the continent, impacting both consumer 
        spending and industrial production.
        """
        
        # Generate the summary
        summarizer_tool.summarize_hindi(article_to_test)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n--- Test Interrupted ---")

ModuleNotFoundError: No module named 'summarizer_tool'